# Modelos de Árboles de decisión

## 1. Random Forest

### Descripción

Random Forest es un modelo de aprendizaje supervisado basado en árboles de decisión que emplea el principio de ensamble para mejorar la precisión y la robustez de las predicciones. Este enfoque se utiliza principalmente en datos tabulares y es una de las técnicas más populares y eficientes en tareas de clasificación y regresión.

Además, este modelo pertenece a la categoría de modelos de tipo bagging  ya que emplea una combinación de múltiples árboles de decisión entrenados de manera independiente y al final se elige la solución mayoritaria o el promedio de las predicciones. Por lo que Random Forest al permitir emplear múltiples árboles de decisión entrenados de distinta manera, permitiendo reducir el sobreajuste y mejorar la generalización. 


### Implementación


En este proyecto vamos a emplear el primer preprocesado realizado, a continuación se llevará a cabo la técnica de submuestreo para balancear el conjunto de los datos. Pese a que se haya probado otra técnica como SMOTE para balancear los datos, pese a que no porporcionaba buenas métricas, se ha decantado por el uso del submuestreo. 


In [12]:
import pandas as pd

# Cargar el dataset de entrenamiento con el primer preprocesado
df = pd.read_csv('../../../data/processed/df_train.csv')

df

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,2005-11-01,2006,2,...,0,2,0,1,0.0,0.0,2005-12-31,8000.0,0.0,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,2005-06-06,2005,2,...,1,2,1,1,0.0,0.0,2005-07-31,166000.0,0.0,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,2003-03-18,2003,2,...,4,2,1,2,1.0,0.0,2003-03-31,25000.0,0.0,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,1995-06-28,1995,2,...,0,0,1,0,0.0,0.0,1996-01-31,220100.0,0.0,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,2009-04-16,2009,0,...,0,0,0,1,0.0,0.0,2009-05-31,25000.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,4f9443d2a46,1573725008,"SIBILA RACE ENGINEERING, INC",MASSILLON,OH,CITIZENS BANK NATL ASSOC,RI,2005-12-09,2006,1,...,0,1,0,1,0.0,0.0,2005-12-31,70000.0,0.0,1
22831,798db2753a7,2011184008,ENVIRO SHIELD POWER WASHING,SPRINGBORO,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,1998-04-27,1998,2,...,0,0,1,0,0.0,1.0,1998-05-31,30000.0,0.0,1
22832,ddb3c5e9bff,4082983001,"MAINLINE TRCK&TRAILR SRVC, INC",BEDFORD,OH,GROWTH CAPITAL CORP.,OH,1990-05-09,1990,16,...,6,10,1,0,0.0,0.0,1991-02-13,92000.0,0.0,1
22833,407200a5dfe,7783283010,TIN BOX STUDIO,CINCINNATI,OH,KEYBANK NATIONAL ASSOCIATION,OH,1994-11-10,1995,1,...,0,0,1,0,0.0,1.0,1995-01-31,20000.0,0.0,1


Con esos datos preprocesados primeramente, antes de entrenar al modelo realizamos el balanceo usando submuestreo y además, seleccionamos que columnas no van a participar en el proceso de entrenamieto. Se ha decidido no usar esas columnas, ya que de todas las características analizadas, eran las que menos información podían aportar a la hora de clasificar la concesión del crédito. 

En una primera idea se realizó el entrenamiento con todas las hipótesis planteadas, sin embargo, pese al mal funcionamiento de los modelos de árboles de decisión, se cambió la estrategiay se emplea esta estrategia de seleccionar todas las columnas y descartar las que se cree que no aporta información relevante.

En este preprocesado adicional, también se lleva a cabo la codificación de las variables categóricas y se rellenan las variables nulas.

In [13]:
# Columnas que no aportan información
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

df_clean.fillna(0, inplace=True)


# Balancear el DataFrame realizando submuestreo 
# En la columna Accept, el valor 0 corresponde con la clase minoritaria y el 1 la mayoritaria.
# Es decir, hay mayor cantidad de datos con préstamos aceptados que rechazados

# Extraemos los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizamos un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases y mezclamos los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

División del dataset para entrenar el modelo

In [14]:
from sklearn.model_selection import train_test_split

X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


### Configuración de hiperparámetros de Random Forest y entrenamiento

Para llevar a cabo la configuración de los hiperparámetros hemos tenido en cuenta lo la documentación del modelo. Para el criterio de división empleado, se utiliza Gini ya que proporciona una solución rápida y eficiente en la construcción de los árboles. Además, en cuanto al número de estimadores y la profundidad máxima, es decir, como de grande va a ser nuestro bosque y como de profundo va a ser cada árbol se ha optado por valores más altos. El número de estimadores es elevado, ya que como cada árbol de forma individual tiene que dar su valoración, si se poseen gran cantidad de árboles, el error individual promedio disminuye, y por lo tanto, es más robusto pero tiene mayor consumo computacional. Por otro lado, la profundidad del árbol mide la cantidad de decisiones que tiene que realizar cada árbol para tomar una decisión, por lo que se ha fijado en un valor alto, debido a que el conjunto de datos contiene gran cantidad de características, no puede ser muy general porque no se termina de entrenar bien los datos ni muy específico ya que si no se tiende al sobreajuste [1]. Cabe destacar que para seleccionar los parámetros más óptimos y tener la certeza de que eran los adecuados, se ha empleado GridSearchCV.

In [15]:
from sklearn.ensemble import RandomForestClassifier

class_weights = {0: 5, 1: 2}  # Ajusta estos valores según tus datos
rf = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',
    class_weight=class_weights,
    random_state=42
)

rf.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 5, 1: 2}, max_depth=80,
                       n_estimators=500, random_state=42)

### Evaluación

In [16]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

y_pred = rf.predict(X_test)

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

# Reporte de Clasificación (precision, recall, f1-score, etc.)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calculamos el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")

Matriz de Confusión:
[[592 175]
 [286 480]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.67      0.77      0.72       767
           1       0.73      0.63      0.68       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.70


A continuación, tenemos la comprobación con GridSearch

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, make_scorer

# Cargamos el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')

# Columnas que no aportan información
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

df_clean.fillna(0, inplace=True)


# Balancear el DataFrame realizando submuestreo 
# En la columna Accept, el valor 0 corresponde con la clase minoritaria y el 1 la mayoritaria.
# Es decir, hay mayor cantidad de datos con préstamos aceptados que rechazados

# Extraemos los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizamos un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases y mezclamos los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

# Separar X, Y
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# Dividimos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Definir el grid de hiperparámetros para RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 400, 500],
    'max_depth': [None, 10, 20, 40, 80],
    'max_features': ['sqrt', 'log2', None]
}

# Crear un scorer basado en el Macro F1-Score para evaluar el balance entre las clases
scorer = make_scorer(f1_score, average='macro')

# 5. Configurar y ejecutar GridSearchCV
grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight={0: 5, 1: 2}, n_jobs=10),
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1
)
grid_rf.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_rf.best_params_)

# 6. Evaluar el mejor modelo en el conjunto de prueba
best_rf = grid_rf.best_estimator_
y_pred = best_rf.predict(X_test)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Mejores parámetros encontrados:
{'max_depth': 80, 'max_features': 'sqrt', 'n_estimators': 500}
Matriz de Confusión:
[[592 175]
 [286 480]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.67      0.77      0.72       767
           1       0.73      0.63      0.68       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.6976687079820924


## 2. XGBoost

### Descripción
Este modelo también se trata de un algoritmo de aprendizaje supervisado basado en el enfoque de boosting, empleando tamién para las tareas de clsificación y regresión. A diferencia del Random Forest,  XGBoost, se encarga de construir los árboles de manera secuencial, y cada árbol va a intentar corregir los errores producidos por el árbol precedente.

### Implementación

La implementación es muy similar a la utilizada en el Random Forest, a diferencia de la configuracion de los hiperparámetros.

Para la configuración, en el caso de XGBoost, el número de estimadores y la profundidad máxima de los árboles se configuraron de manera similar a los valores de Random Forest, siguiendo las recomendaciones de la documentación de XGBoost, y lo indicado por GridSearchCV. Además de estos hiperparámetros, se utilizó la métrica logloss para la evaluación, ya que es especialmente adecuada para problemas de clasificación binaria. Esta métrica penaliza las predicciones incorrectas, lo que mejora la precisión del modelo. También se ha configurado la tasa de aprendizaje del modelo es baja, de tal manera que va a permitir que vaya aprendiendo de forma gradual y sea más generalizable, evitando tender al sobreajuste, y en cada árbol al usar datos con tantas características, se va a emplear un 60% de estas distribuidas de forma aleatoria. [2] [3]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# Cargamos el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')


# Columnas que no aportan información
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

df_clean.fillna(0, inplace=True)


# Balancear el DataFrame realizando submuestreo 
# En la columna Accept, el valor 0 corresponde con la clase minoritaria y el 1 la mayoritaria.
# Es decir, hay mayor cantidad de datos con préstamos aceptados que rechazados

# Extraemos los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizamos un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases y mezclamos los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

#Separamos X, Y
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# Dividimos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Configuración y entrenamiento del modelo
# Se asigna un peso de 5 a la clase 0 (créditos rechazados) y 2 a la clase 1 (créditos aceptados)
sample_weights = y_train.map({0: 5, 1: 2})

# Inicializamos y entrenamos el modelo XGBoost
xgb = XGBClassifier(
    n_estimators=200,
    max_depth=80,
    learning_rate=0.05,
    eval_metric='logloss',
    colsample_bytree = 0.6,
    random_state=42
)
xgb.fit(X_train, y_train, sample_weight=sample_weights)

# Realizamos predicciones en el conjunto de prueba
y_pred = xgb.predict(X_test)

# Evaluamos el modelo

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

# Reporte de Clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")


Matriz de Confusión:
[[588 179]
 [261 505]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.77      0.73       767
           1       0.74      0.66      0.70       766

    accuracy                           0.71      1533
   macro avg       0.72      0.71      0.71      1533
weighted avg       0.72      0.71      0.71      1533

Macro F1-Score: 0.71


A continuación, tenemos la comprobación con GridSearch

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# Cargamos el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')

cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Codificar variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame realizando submuestreo 
# En la columna Accept, el valor 0 corresponde con la clase minoritaria y el 1 la mayoritaria.
# Es decir, hay mayor cantidad de datos con préstamos aceptados que rechazados

# Extraemos los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizamos un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases y mezclamos los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

#Separamos X, Y
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# Dividimos el dataset de forma estratificada (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

sample_weights = y_train.map({0: 5, 1: 2})

# Definimos el modelo XGBoost
xgb = XGBClassifier(
    eval_metric='logloss',
    random_state=42
)

# Definimos el rango de parámetros para GridSearch
param_grid = {
    'n_estimators': [100, 200, 300],          # Número de árboles
    'max_depth': [3, 5, 6, 10, 50, 80],               # Profundidad de los árboles
    'learning_rate': [0.01, 0.05, 0.1],       # Tasa de aprendizaje
    'colsample_bytree': [1, 0.6, 0.8]
}

# Realizamos GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1, scoring='f1_macro')
grid_search.fit(X_train, y_train, sample_weight=sample_weights)

# Imprimimos los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Usamos el mejor modelo encontrado por GridSearch
best_xgb = grid_search.best_estimator_

# Realizamos predicciones en el conjunto de prueba
y_pred = best_xgb.predict(X_test)

# Evaluamos el modelo

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

# Reporte de Clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")


Fitting 3 folds for each of 162 candidates, totalling 486 fits
Mejores parámetros encontrados:
{'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_depth': 80, 'n_estimators': 100}
Matriz de Confusión:
[[608 159]
 [273 493]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.79      0.74       767
           1       0.76      0.64      0.70       766

    accuracy                           0.72      1533
   macro avg       0.72      0.72      0.72      1533
weighted avg       0.72      0.72      0.72      1533

Macro F1-Score: 0.72


## Ensemble del Random Forest y XGBoost

Finalmente con estos dos modelos hemos realizado un ensemble, que va a permitir realizar un aprendizaje automático conjunto, para mejorar la precisión y robustez del modelo, en lugar de emplear únicamente uno de los dos modelos mencionados. 

Para ello, se estudió dos tipos de ensemble:
1. Voting Classifier
2. Stacking Classifier

Después de un análisis de los dos, el que mejor funcionaba para nuestro conjunto de datos se trataba del Voting Ensemble

### Voting Classifier 

Esta técnica combina las predicciones de ambos modelos. Se puede elegir entre:

* Hard Voting: Cada modelo vota por una clase y se toma la mayoría.

* Soft Voting: Se promedian las probabilidades de cada clase y se selecciona la de mayor probabilidad final.

[4]

#### Implementación con Voting Classifier:

La implementación es similar a los dos modelos anteriores con las configuraciones ya establecidas. Cabe resaltar que hemos obtado por una votación de tipo suave, la cuál es menos agresiva y clasifica mejor los datos.

In [22]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Cargar el dataset
df = pd.read_csv('../../../data/processed/df_train.csv')

# Eliminamos columnas irrelevantes
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Codificamos variables categóricas a dummy
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

df_clean.fillna(0, inplace=True)

# Balancear el DataFrame realizando submuestreo 
# En la columna Accept, el valor 0 corresponde con la clase minoritaria y el 1 la mayoritaria.
# Es decir, hay mayor cantidad de datos con préstamos aceptados que rechazados

# Extraemos los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizamos un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases y mezclamos los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

#Separamos X, Y
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Definimos los dos modelos
rf = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',
    class_weight={0:5, 1:2},
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth= 80,
    learning_rate=0.05,
    eval_metric='logloss',
    random_state=42
)

# Combinamos los modelos con VotingClassifier (voting='soft' para promediar probabilidades)
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb_model)],
    voting='soft', 
    n_jobs=-1
)

# Entrenamos el ensemble
voting_clf.fit(X_train, y_train)

# Evaluamos el ensemble
y_pred = voting_clf.predict(X_test)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Matriz de Confusión:
[[573 194]
 [252 514]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.75      0.72       767
           1       0.73      0.67      0.70       766

    accuracy                           0.71      1533
   macro avg       0.71      0.71      0.71      1533
weighted avg       0.71      0.71      0.71      1533

Macro F1-Score: 0.7086356136176132


## Conclusión

El Voting Classifier que combina RandomForest y XGBoost, demuestra un rendimiento sólido con un "accuracy" de 0.71 y un Macro F1-Score de 0.71, siendo el modelo más robusto de los 3 analizados. Una accuracy de 0.71 implica la proporción de predicciones correctas sobre el total de las predicciones realizadas, por lo que para el 71% de los datos del conjunto se clasifican correctamente. Además, la macro F1-score, nos indica que el modelo tiene buen desempeño tanto para identificar correctamente los positivos (recall) como la precisión de las predicciones. Si analizamos en profundidad, este modelo presenta un recall del 75% para los prestamos rechazados y un 67% para los préstamos aceptados, lo que refleja un buen desempeño  en la clasificación de la clase minotoria, gracias al balanceo de los datos. El modelo ha aprendido a categorizar adecuadamente los préstamos rechazados, mejorando su capacidad de predección para esta clase.

Al comparar los modelos individuales a través de la validación cruzada, se observa que para estos tres modelos, las métricas obtenidas son bastante similares, indicando que los modelos son estables y robustos.

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost as xgb
import pandas as pd

# Cargar el dataset
df = pd.read_csv('../../../data/processed/df_train.csv')

# Preprocesado
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Convertir variables categóricas a dummy
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar valores nulos
df_clean.fillna(0, inplace=True)

# Balanceo del DataFrame
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# Separar features y target
X = df_balanced.drop('Accept', axis=1)
y = df_balanced['Accept']

# Definir modelos
rf = RandomForestClassifier(n_estimators=500, max_depth=80, max_features='sqrt', class_weight={0:5, 1:2}, random_state=42)
xgb_model = xgb.XGBClassifier(n_estimators=200, max_depth=80, learning_rate=0.05, eval_metric='logloss', colsample_bytree=0.6, random_state=42)
voting_clf = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb_model)], voting='soft', n_jobs=-1)

# Validación cruzada para Random Forest
rf_scores = cross_val_score(rf, X, y, cv=5)
print(f"Random Forest - Cross-validation score mean: {rf_scores.mean():.4f}")

# Validación cruzada para XGBoost
xgb_scores = cross_val_score(xgb_model, X, y, cv=5)
print(f"XGBoost - Cross-validation score mean: {xgb_scores.mean():.4f}")

# Validación cruzada para el VotingClassifier
voting_scores = cross_val_score(voting_clf, X, y, cv=5)
print(f"VotingClassifier - Cross-validation score mean: {voting_scores.mean():.4f}")


Random Forest - Cross-validation score mean: 0.7130
XGBoost - Cross-validation score mean: 0.6973
VotingClassifier - Cross-validation score mean: 0.7073


## Referencias

- [RandomForestClassifier - Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
- [XGBoost Parameters - XGBoost Documentation](https://xgboost.readthedocs.io/en/release_3.0.0/parameter.html)
- [XGBoost Hyperparameters - AWS SageMaker](https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/xgboost_hyperparameters.html)

- [VotingClassifier - Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)